<a href="https://colab.research.google.com/github/vishalmysore/AI/blob/main/examples/modelwar/Modelwars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

In [5]:
from transformers import GPTNeoForCausalLM, AutoTokenizer,AutoModel
import torch
def print_board(board):
    for row in board:
        print(" | ".join(row))
        print("---------")

def check_winner(board):
    # Check rows
    for row in board:
        if all(cell == 'X' for cell in row) or all(cell == 'O' for cell in row):
            return True

    # Check columns
    for col in range(3):
        if all(board[row][col] == 'X' for row in range(3)) or all(board[row][col] == 'O' for row in range(3)):
            return True

    # Check diagonals
    if all(board[i][i] == 'X' for i in range(3)) or all(board[i][2 - i] == 'O' for i in range(3)):
        return True

    return False

def is_board_full(board):
    return all(all(cell != ' ' for cell in row) for row in board)

def play_game(model1, model2):
    player1 = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-125m', from_flax=True)
    tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125m', use_fast=True)
    player2 = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-125m', from_flax=True)

    board = [[' ' for _ in range(3)] for _ in range(3)]
    current_player = 'X'

    while True:
        print_board(board)

        if current_player == 'X':
            print("Player 1's turn (X)")
            move = get_model_move(player1, tokenizer, board)
        else:
            print("Player 2's turn (O)")
            move = get_model_move(player2, tokenizer, board)

        row, col = move // 3, move % 3

        if board[row][col] == ' ':
            board[row][col] = current_player
            if check_winner(board):
                print_board(board)
                print(f"Player {current_player} wins!")
                break
            elif is_board_full(board):
                print_board(board)
                print("It's a tie!")
                break
            else:
                current_player = 'O' if current_player == 'X' else 'X'
        else:
            print("Invalid move. Try again.")

def get_model_move(model, tokenizer, board):
    flat_board = [cell for row in board for cell in row]
    input_text = f"The current Tic Tac Toe board is: {', '.join(flat_board)}. Your move? "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    with torch.no_grad():
        output = model.generate(input_ids, max_length=2)
        print(tokenizer.decode(output[0], skip_special_tokens=True))
        move = int(tokenizer.decode(output[0], skip_special_tokens=True))

    return move

if __name__ == "__main__":
    model1 = "gpt2"
    model2 = "gpt2"

    play_game(model1, model2)


All Flax model weights were used when initializing GPTNeoForCausalLM.

Some weights of GPTNeoForCausalLM were not initialized from the Flax model and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All Flax model weights were used when initializing GPTNeoForCausalLM.

Some weights of GPTNeoForCausalLM were not initialized from the Flax model and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  |   |  
---------
  |   |  
---------
  |   |  
---------
Player 1's turn (X)
The current Tic Tac Toe board is: , , , , , , , , . Your move?  


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 32, but `max_length` is set to 2. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


ValueError: invalid literal for int() with base 10: 'The current Tic Tac Toe board is: , , , , , , , , . Your move?  '